<a href="https://colab.research.google.com/github/HigherGround189/EGT309-Team-Harish-Kanna/blob/main/EDA/AI_Solution_Development_Case_Study_Ying_Ray_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup


In [ ]:
!gdown --fuzzy https://drive.google.com/file/d/17S8vGBsbaAcuxgwSZLGhOOrrhfaqio7j/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=17S8vGBsbaAcuxgwSZLGhOOrrhfaqio7j
To: /content/bmarket.db
100% 3.15M/3.15M [00:00<00:00, 104MB/s]


In [ ]:
import polars as pl
import sqlite3
import plotly.express as px

In [ ]:
conn = sqlite3.connect('/content/bmarket.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
table_name = cursor.fetchall()[0][0]
print(table_name)

bank_marketing


In [ ]:
pl.Config(set_tbl_cols=-1, fmt_str_lengths=65535, set_tbl_width_chars=65535)

def lprint(dataframe, num=-1):
    with pl.Config(tbl_rows=num):
        print(dataframe)

df = pl.read_database(query=f"SELECT * FROM {table_name}", connection=conn)
print(df)


shape: (41_188, 12)
┌───────────┬──────────┬─────────────┬────────────────┬─────────────────────┬────────────────┬──────────────┬───────────────┬────────────────┬────────────────┬───────────────────────┬─────────────────────┐
│ Client ID ┆ Age      ┆ Occupation  ┆ Marital Status ┆ Education Level     ┆ Credit Default ┆ Housing Loan ┆ Personal Loan ┆ Contact Method ┆ Campaign Calls ┆ Previous Contact Days ┆ Subscription Status │
│ ---       ┆ ---      ┆ ---         ┆ ---            ┆ ---                 ┆ ---            ┆ ---          ┆ ---           ┆ ---            ┆ ---            ┆ ---                   ┆ ---                 │
│ i64       ┆ str      ┆ str         ┆ str            ┆ str                 ┆ str            ┆ str          ┆ str           ┆ str            ┆ i64            ┆ i64                   ┆ str                 │
╞═══════════╪══════════╪═════════════╪════════════════╪═════════════════════╪════════════════╪══════════════╪═══════════════╪════════════════╪══════════════

In [ ]:
# Get distribution of each column
# Get percentage of each unique value that subscribes

def col_distribution(df, col):
    fig = px.histogram(df, x=col, color=col)
    fig.show()

In [ ]:
processed_df = df.with_columns(
    pl.col("Age").str.extract(r"\d+", group_index=0).cast(pl.Int64)
).filter(pl.col("Age") != 150)

col_distribution(processed_df, "Age")


# Analysis Per Column

# Client ID

## Age

## Occupation

## Marital Status

## Credit Default

## Housing Loan

## Personal Loan

## Contact Method

## Campaign Calls

## Previous Contact Days

## Subscription Status